In [2]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
from functools import partial
import Evaluation_func as func
from tqdm import tqdm_notebook as tqdm
from datetime import date, timedelta

In [2]:
df = pd.read_csv(r'D:\Customer_Value\data\evaluate\st_sample_test_large.csv')

Collect History Data

In [3]:
df_list = [group[1] for group in df.groupby(df['S_IDNO'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.pick_history_data, time_period=120), df_list), total=len(df_list)), 1):
            output_list.append(x)
            
df_history = pd.concat(output_list, axis=0)

In [4]:
del df

Combine with asset data

In [5]:
df_list = [group[1] for group in df_history.groupby(df_history['YEAR'])]
output_list = []

for i, d in enumerate(tqdm(df_list, total=len(df_list)), 1):
    year = d['YEAR'].iloc[0]
    asset = pd.read_csv(f'D:\\Customer_Value\\data\\asset\\st_asset{int(year) - 2012}_processed.csv', converters={'S_IDNO': str, 'MTH_DATE': str})
    d = pd.merge(d, asset, on=['S_IDNO', 'MTH_DATE'], how='left')
    output_list.append(d)
    
df_history = pd.concat(output_list, axis=0)

Asset Encoding

In [4]:
reference_dict = {'AMT_B': {1: 0,
 20000: 1,
 35000: 2,
 50000: 3,
 70000: 4,
 100000: 5,
 140000: 6,
 200000: 7,
 310000: 8,
 600000: 9},
'AMT_S': {1: 0,
 22000: 1,
 38000: 2,
 55000: 3,
 80000: 4,
 110000: 5,
 150000: 6,
 230000: 7,
 360000: 8,
 700000: 9},
'ST_ASSET': {1: 0,
 12000: 1,
 50000: 2,
 100000: 3,
 200000: 4,
 310000: 5,
 500000: 6,
 800000: 7,
 1350000: 8,
 2800000: 9},
'MP_ASSET': {1: 0,
 35000: 1,
 70000: 2,
 110000: 3,
 170000: 4,
 240000: 5,
 350000: 6,
 550000: 7,
 950000: 8,
 2150000: 9},
'SS_ASSET': {1: 0,
 40000: 1,
 75000: 2,
 110000: 3,
 170000: 4,
 240000: 5,
 350000: 6,
 500000: 7,
 800000: 8,
 1650000: 9}}

In [5]:
df_list = [group[1] for group in df_history.groupby(df_history['sample_no'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.label_asset, reference_dict=reference_dict), df_list), total=len(df_list)), 1):
            output_list.append(x)
            
df = pd.concat(output_list, axis=0)

Save Data

In [6]:
df.to_csv(r'D:\Customer_Value\data\evaluate\st_sample_all_large.csv', index=False)